# Dashboard

### Imports

In [1]:
%run "Imports.ipynb"

In [2]:
# Record start time
start_time = time.time()

In [3]:
%run "Utilities.ipynb"
%run "NewClasses.ipynb"
%run "Newsimulation_functions.ipynb"

Code was last run on: 2023-09-16


In [4]:
# Delay
try:
    pause_code('2023-09-13T18:20:00')
except:
    pass

-225792.046315


In [5]:
# This reads in Chadwick register with player codes.
keep_list = ['key_mlbam', 'key_fangraphs', 'key_bbref_minors', 'key_bbref', 'name_first', 'name_last']
chadwick = read_chadwick(keep_list)

In [6]:
# This reads in a map of team name, codes, and the shorthand MLB uses in their URLs
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))

# We just need teams right now
team_map = team_map[['FULLNAME', 'BBREFTEAM', 'MLBURL', 'FANGRAPHSTEAM', 'VENUE_ID', 'SFBBTEAM', 'DKTEAM', 'ROTOWIRETEAM', 'FANPROSTEAM']]

In [7]:
crosswalk = pd.read_csv(os.path.join(baseball_path, "Utilities", "Player Crosswalk.csv"), encoding='iso-8859-1') 
crosswalk = crosswalk[['mlbamid', 'steamerid']]
crosswalk.rename(columns={'mlbamid':'mlbamid_fill'}, inplace=True)

In [8]:
# Read in history file        
history = pd.read_csv(os.path.join(baseball_path, "Utilities", "Contests.csv"))

# Sort by date, then draft group, then fee
history.sort_values(['date', 'draftGroupId', 'entryFee'], ascending=False)
# Keep only one observation per draft group 
history.drop_duplicates('draftGroupId', keep='first', inplace=True)

history_full = history.copy()
history = history.query('result == 1').query('payout == 1').query('salary == 1')


history = history.reset_index(drop=True)
history['date'] = history['date'].astype('int').astype('str')

### Import Contests

In [9]:
# Choose contests of interest
def daily_contests(date=todaysdate):
    df = pd.read_csv(os.path.join(baseball_path, "2. Contests", "A. Daily Contests", f"Contests {date}.csv"))
    
    # Create an empty list to store the results
    results = []

    # Iterate over the rows of the dataframe
    for index, row in df.iterrows():
        name = row["Name"]
        if "Four-Seamer" in name and "vs" not in name and "Turbo" not in name:
            contest_key = row["contestKey"]
            draft_group_id = row["draftGroupId"]
            contestTime = row['contestTime']
            results.append({"Name": name, "contestKey": contest_key, "draftGroupId": draft_group_id, "contestTime": contestTime})

    # Convert the results to a dataframe
    results_df = pd.DataFrame(results)

    return results_df

In [10]:
todays_contests = daily_contests(todaysdate)
todays_contests

,Name,contestKey,draftGroupId,contestTime
0,MLB $2K Four-Seamer (Early),150491051,92657,Sat 2:10PM
1,MLB $4K Four-Seamer [20 Entry Max],150491095,92411,Sat 7:05PM
2,MLB $1K Four-Seamer (Night),150491140,92416,Sat 9:07PM


In [11]:
# Select contest to enter
contest = 1

# Contest info
Name = todays_contests['Name'][contest]
contestKey = todays_contests['contestKey'][contest]
draftGroupId = todays_contests['draftGroupId'][contest]
contestTime = todays_contests['contestTime'][contest]

print(Name, contestKey, draftGroupId, contestTime)

MLB $4K Four-Seamer [20 Entry Max] 150491095 92411 Sat 7:05PM


### 5. Depth Charts

In [12]:
%run "05. Depth Charts.ipynb"

In [13]:
create_depth_charts(todaysdate, todaysdate)

### 6. Rosters

In [14]:
%run "06. Rosters.ipynb"

In [15]:
# List of games between dates
games = statsapi.schedule(start_date=todaysdate_slash, end_date=todaysdate_slash)
    
# Loop over games to create rosters
output = Parallel(n_jobs=-2, verbose=5)(delayed(create_rosters)(x=game, team_map=team_map) for game in games)

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 15 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of  16 | elapsed:   17.0s remaining:  1.2min
[Parallel(n_jobs=-2)]: Done   7 out of  16 | elapsed:   17.9s remaining:   23.1s
[Parallel(n_jobs=-2)]: Done  11 out of  16 | elapsed:   18.1s remaining:    8.2s
[Parallel(n_jobs=-2)]: Done  16 out of  16 | elapsed:   28.7s finished


### 7. Stats

In [16]:
%run "07. Stats.ipynb"

In [17]:
scrape_batters()
scrape_pitchers()

In [18]:
%%time
run_stats(date=todaysdate, historic=False)

CPU times: total: 2.05 s
Wall time: 51.1 s


### 8. Weather

In [19]:
%run "08. Weather.ipynb"

In [20]:
swish = scrape_swishanalytics(team_map)
roto = scrape_rotogrinders()
roto

,Tag,Tag2,Matchup,Description
0,GREEN,None,CIN@NYM,- Winds out to right at 10mph.
1,YELLOW,GREEN,MIN@CHW,"- For now models have all the games staying just north of Chicago. It's close enough to revisit, but I'm optimistic."
2,ORANGE,YELLOW,HOU@KCR,"- Storm line moves in right around gametime, but it also moves out relatively quickly. I feel fairly strongly that we'll see a delay at some point (maybe they'll late start it), but I also think that they'll be able to play eventually."
3,YELLOW,None,PHI@STL,"- The coverage of the storms models are showing in STL isn't quite to the extent of those in KC, but there's still a chance of a delay."


In [21]:
ballparkpal = scrape_ballparkpal(todaysdate_dash)

### 9. Matchups 

In [22]:
%run "09. Matchups.ipynb"

In [23]:
lineups, projected = scrape_lineups(team_map, date=todaysdate_dash, early=False)
print("Projected lineups : ", projected)

Projected lineups :  ['SF', 'COL', 'BOS', 'TOR', 'SD', 'OAK', 'ATL', 'MIA', 'TEX', 'CLE', 'NYY', 'PIT', 'TB', 'BAL', 'CIN', 'NYM', 'MIN', 'CHW', 'HOU', 'KC', 'WAS', 'MIL', 'PHI', 'STL', 'CHC', 'ARI', 'DET', 'LAA', 'LAD', 'SEA']


In [24]:
run_matchups(team_map, contestKey=contestKey, date=todaysdate, backtest=False, early=False, historic=False)

['WSN_MIL 09162023 0710PM ET', 'PHI_STL 09162023 0715PM ET', 'TBR_BAL 09162023 0705PM ET', 'LAD_SEA 09162023 0940PM ET', 'HOU_KCR 09162023 0710PM ET', 'MIN_CHW 09162023 0710PM ET', 'CHC_ARI 09162023 0810PM ET', 'CIN_NYM 09162023 0710PM ET', 'DET_LAA 09162023 0907PM ET', 'SFG_COL 09162023 0810PM ET']
2023-09-16
TBR_BAL 09162023 0705PM ET
WSN_MIL 09162023 0710PM ET
HOU_KCR 09162023 0710PM ET
MIN_CHW 09162023 0710PM ET
CIN_NYM 09162023 0710PM ET
PHI_STL 09162023 0715PM ET
CHC_ARI 09162023 0810PM ET
SFG_COL 09162023 0810PM ET
DET_LAA 09162023 0907PM ET
LAD_SEA 09162023 0940PM ET


### 10. Sims

In [66]:
%run "10. Sims.ipynb"
# This reads in a map of team name, codes, and the shorthand MLB uses in their URLs
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))

# We just need teams right now
team_map = team_map[['FULLNAME', 'BBREFTEAM', 'MLBURL', 'FANGRAPHSTEAM', 'VENUE_ID', 'SFBBTEAM', 'DKTEAM', 'ROTOWIRETEAM', 'FANPROSTEAM']]

In [67]:
%run "NewClasses.ipynb"
%run "Newsimulation_functions.ipynb"

Code was last run on: 2023-09-16


In [68]:
model_binary, model_outs, model_safe, pull, dp_rates, dp_base_rates, advances, late_list, broken_list = sim_imports()

In [69]:
weather_df = create_weather_df(todaysdate)

In [70]:
DKSalaries, matchup_list, matchup_path, opener_list = create_matchup_list(contestKey, date, late_list, late=False)

In [63]:
matchup = matchup_list[0]
matchup

'CHC_ARI 09162023 0810PM ET.xlsx'

In [64]:
# team_map.set_index('BBREFTEAM', inplace=True)

In [50]:
matchup_folder = "Matchups" + str(contestKey)
matchup_path = os.path.join(baseball_path, "9. Matchups", "B. Matchups", matchup_folder)

all_scores_list=[]
game_template = create_game(matchup, matchup_path, team_map, weather_df)

game_template.home_batters[10].Name
game_template.home_batters[10].order

-99

In [51]:
game = sim_game(game_template, pull, model_binary, model_outs, model_safe, dp_rates, dp_base_rates, advances, opener_list)
game


In [52]:
game.home_score

3

In [56]:
player_sims, score_df = create_matchup_df(matchup, DKSalaries, weather_df, date, contestKey, opener_list,
            team_map, pull, model_binary, model_outs, model_safe, dp_rates, dp_base_rates,
            advances, num_sims=1000)

In [72]:
%%time
model_binary, model_outs, model_safe, pull, dp_rates, dp_base_rates, advances, late_list, broken_list = sim_imports()

run_sims(team_map=team_map, pull=pull, model_binary=model_binary, model_outs=model_outs, model_safe=model_safe, 
         dp_rates=dp_rates, dp_base_rates=dp_base_rates, advances=advances, broken_list=broken_list, 
         date=todaysdate, contestKey=contestKey, last_list=[], num_sims=1000, late=False, historic=False)

['TBR_BAL 09162023 0705PM ET.xlsx', 'CIN_NYM 09162023 0710PM ET.xlsx', 'HOU_KCR 09162023 0710PM ET.xlsx', 'MIN_CHW 09162023 0710PM ET.xlsx', 'WSN_MIL 09162023 0710PM ET.xlsx', 'PHI_STL 09162023 0715PM ET.xlsx', 'CHC_ARI 09162023 0810PM ET.xlsx', 'SFG_COL 09162023 0810PM ET.xlsx', 'DET_LAA 09162023 0907PM ET.xlsx', 'LAD_SEA 09162023 0940PM ET.xlsx']
TBR_BAL 09162023 0705PM ET.xlsx
Home Win Rate: 0.518
Mean Away Score: 3.891
Mean Home Score: 3.837
Mean Total: 7.728
Median Total: 7.0

CIN_NYM 09162023 0710PM ET.xlsx
Home Win Rate: 0.47
Mean Away Score: 4.213
Mean Home Score: 3.768
Mean Total: 7.981
Median Total: 7.0

HOU_KCR 09162023 0710PM ET.xlsx
Home Win Rate: 0.616
Mean Away Score: 3.754
Mean Home Score: 4.929
Mean Total: 8.683
Median Total: 8.0

MIN_CHW 09162023 0710PM ET.xlsx
Home Win Rate: 0.564
Mean Away Score: 4.422
Mean Home Score: 4.81
Mean Total: 9.232
Median Total: 9.0

WSN_MIL 09162023 0710PM ET.xlsx
Home Win Rate: 0.57
Mean Away Score: 4.076
Mean Home Score: 4.709
Mean Tota

### 11. Projections

In [ ]:
%run "11. Projections.ipynb"

### Gambling

In [ ]:
gamble = daily_gamble(todaysdate, team_map)

### Daily

In [ ]:
slate = pick_slate(Name)
slate

#### RotoWire

In [ ]:
# Read in slates
roto_slates = pd.read_csv(os.path.join(baseball_path, "11. Projections", "RotoWire", "A. Slates", f"Slates {todaysdate}.csv"))

# List of slates to keep
slates_to_keep = ["All", "Early", "Afternoon", "Night", "Late Night"]
roto_slates = roto_slates[roto_slates['name'].isin(slates_to_keep)]

# Scrape projections
for slateID in roto_slates['slateID']:
    rotowire_points = roto_projections(slateID)

In [ ]:
# Choose index for current slate
roto_index = roto_slates[roto_slates['name'] == slate].index[0]
# Identify code
roto_slate = roto_slates.loc[roto_index, 'slateID']

# Read in projections
roto_projections = pd.read_csv(os.path.join(baseball_path, "11. Projections", "RotoWire", "B. Projections", f"Slate {roto_slate}.csv"), encoding='iso-8859-1')
roto_projections.head()

#### DFF

In [ ]:
# Read in slates
dff_slates = pd.read_csv(os.path.join(baseball_path, "11. Projections", "DFF", "A. Slates", f"Slates {todaysdate}.csv"))

# Scrape projections
for code in dff_slates['URL']:
    dff_projections(code)

In [ ]:
# Choose index for current slate
dff_index = dff_slates[dff_slates['Slate Type'] == slate].index[0]
# Identify code
dff_slate = dff_slates.loc[dff_index, 'URL']

# Read in projections
dff_projections = pd.read_csv(os.path.join(baseball_path, "11. Projections", "DFF", "B. Projections", f"Slate {dff_slate}.csv"), encoding='iso-8859-1')
dff_projections.head()

### B1. Evaluate Scores

In [ ]:
run "B1. Evaluate Scores.ipynb"

In [ ]:
gambling = gambling_dashboard(team_map, start_date=todaysdate, end_date=todaysdate)
gambling

### B2. Evaluate Projections

In [ ]:
run "B2. Evaluate Projections.ipynb"

In [ ]:
projections_df = merge_projections(contestKey)
daily = evaluate_projections_pre(projections_df)

### 12. Lineups

In [ ]:
%run "12. Lineups.ipynb"

In [ ]:
%%time
exclude_teams = projected
exclude_players = []

run_lineups(lineups=200, draftGroupId=draftGroupId, contestKey=contestKey, pitcher_max=1, batter_max=1, min1=6.5, min2=0, 
            exclude_teams=exclude_teams, exclude_players=exclude_players, historic=False)

In [ ]:
contestKey

In [ ]:
# Order lineups
choose_lineups(1000, todaysdate, contestKey)

# Create upload file (contains all lineups)
create_upload_file(contestKey)
# Create entry file (contains one lineup per entry)
create_entry_file(draftGroupId, contestKey)

# Print links
print("Lineups: https://www.draftkings.com/lineup/upload")
print("Entries: https://www.draftkings.com/entry/upload")

# Email upload and entry files
email_upload_file(todaysdate, contestKey)

In [ ]:
upload_entries(contestKey)

In [ ]:
# Record end time
end_time = time.time()

# Calculate and print the total execution time
total_time = end_time - start_time
total_time = total_time / 60
print(f"Total execution time: {total_time:.2f} minutes")

# To do:
    1. New Models
    2. Late-swap (pydfs should do most of the heavy lifting)
    3. In-game updates (offseason)
    4. Scrape new contests ($15, double-ups) 
    5. Switch from contest to groupid? (offseason)
    6. Can we replace Chadwick with New Crosswalk?
        If not, we need to archive it, unfortunately, because it regularly doesn't work, doesn't work offline